# Imports

In [0]:
import numpy as np
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings
warnings.filterwarnings('ignore')
import datetime
from collections import Counter
from math import ceil
import numpy as np
import itertools
import sys

# Data Read

In [0]:
# Connection object to Snowflake
dev_url = "https://devkcc.east-us-2.azure.snowflakecomputing.com"
qa_url = "https://qakcc.east-us-2.azure.snowflakecomputing.com"
dev_user = "APP_ADF_PROMOANALYTICS_DEV"
qa_user = "APP_ADF_PROMOANALYTICS_QA"
dev_password = "VSG6eC0bqjSZxkr8"
qa_password = "IgwOidxm4tTmP7dY"
sfOptions = { 
  "sfUrl" : dev_url, 
  "sfUser" : dev_user, 
  "sfPassword" : dev_password, 
  "sfDatabase" : "RGM_ADV_ANLTCS", 
  "sfSchema" : "INT", 
  "sfWarehouse" : "RGM_SALES_GBL_BI_WH"
} 

In [0]:
query = "SELECT * FROM RGM_ADV_ANLTCS.STAGING.V_RGM_LAO_PE_PO_NIS_ADV_NEW1"

In [0]:
data = spark.read.format("snowflake").options(**sfOptions).option("query", query).load().toPandas()

In [0]:
output_files = ['peru_all_data_500', 
               'peru_all_data_500_1000_csv']

hp_table = pd.DataFrame()
for file in output_files:
    df = spark.sql('select * from default.' + file).toPandas()
    df.dropna(inplace = True)
    df = df[df['val_mape'] != float('inf')]
    df[['Customer', 'OSKU']] = df[['Customer', 'OSKU']].astype(str)
    df[['lag', 'ma']] = df[['lag', 'ma']].astype(int)
    df['end_date'] = pd.to_datetime(df['end_date'])
    df['trend'] = df['trend'].str.strip('[]').str.split(', ')
    hp_table = pd.concat([hp_table, df])
    
hp_table.drop('_c0', axis = 1, inplace = True)
hp_table['start_date'] = 0

In [0]:
bl_ratio_sheet = spark.sql('select * from default.peru_new_exp_bl_ratios').toPandas()

# Pre-process Data

In [0]:
data = data.fillna(0)

# Change the data types 
data['CUSTOMER_NAME'] = data['CUSTOMER_NAME'].astype(str)
data['EAN'] = data['EAN'].astype(str)
data['SOLD_TO'] = data['SOLD_TO'].astype(int).astype(str)
data['WEEK'] = data['WEEK'].astype(str)

# Rename columns
data.rename(columns = {'PROMO_INVESTMENT':'PROMO_SPEND', 
                      'SALES_VOLUME': 'QUANTITY_MSU', 
                      'SALES_VALUE': 'NETINV_LC', 
                      'WEEK': 'YEARWEEK', 
                      'SOLD_TO': 'CUSTOMER'}, 
            inplace = True)
numerical_columns = ['QUANTITY_MSU', 'NETINV_LC', 'PROMO_DURATION', 'PROMO_SPEND']
data[numerical_columns] = data[numerical_columns].astype(float)

# Year week columns
data['YEARWEEK'] = [datetime.datetime.strptime(d + '-1', "%Y%W-%w") for d in data['YEARWEEK']]
data['YEAR'] = pd.DatetimeIndex(data['YEARWEEK']).year.astype('category')
data['MONTH'] = pd.DatetimeIndex(data['YEARWEEK']).month.astype('category')

# Relevant Functions

In [0]:
def impute_data(orig_data):
  
  data = orig_data.copy()
  data = data[['YEARWEEK', 'PROMO_SPEND', 'QUANTITY_MSU', 'PROMO_DURATION', 'NETINV_LC']]
  
  # create date array as a timestamp
  #df_date_arr = [datetime.datetime.strptime(d + '-1', "%Y%W-%w") for d in data['YEARWEEK']]
  df_date_arr = data['YEARWEEK']
  sorted_date_arr = sorted(df_date_arr)
  start_date = sorted_date_arr[0]
  end_date = sorted_date_arr[-1]
  
  # calculate actual number of weeks between start and end date
  monday1 = (start_date - datetime.timedelta(days = start_date.weekday()))
  monday2 = (end_date - datetime.timedelta(days = end_date.weekday()))
  master_n_weeks = (monday2 - monday1).days // 7 + 1
  
  master_date_arr = [start_date + datetime.timedelta(days = 7 * x) for x in range(master_n_weeks)]
  
  # group by cal_week and sum
  #data['YEARWEEK'] = [datetime.datetime.strptime(d + '-1', "%Y%W-%w") for d in data['YEARWEEK']]
  
  #data = data.groupby('YEARWEEK').sum()
  #data['YEARWEEK'] = data.index
  #data = data.reset_index(drop = True)
  data['YEARWEEK'] = [d.strftime('%Y-%m-%d') for d in data['YEARWEEK']]
  
  final_df = pd.DataFrame(columns = ['YEARWEEK', 'PROMO_SPEND', 'QUANTITY_MSU', 'PROMO_DURATION', 'NETINV_LC'])
  
  # For every date, check if it is present in the original dataframe. If it isn't impute a row of zeros
  
  for idx, m_date in enumerate(master_date_arr):
    
    m_date = m_date.strftime('%Y-%m-%d')
    new_row = [m_date]
    
    if (m_date not in data['YEARWEEK'].values):
      new_row.extend([[0], 0, [0], 0])
      
    else:
      
      # Overlapping promos with the same start date
      date_rows = data[data['YEARWEEK'] == m_date]
      promo_spend_list = list(date_rows['PROMO_SPEND'])
      promo_dur_list = list(date_rows['PROMO_DURATION'])
      
      #df_idx = [i for i in range(len(data['YEARWEEK'])) if data['YEARWEEK'].values[i] == m_date]
      new_row.extend(list((promo_spend_list, date_rows['QUANTITY_MSU'].values[0], promo_dur_list, date_rows['NETINV_LC'].values[0])))
      #new_row.extend(list(data.loc[df_idx[0], ['PROMO_SPEND', 'QUANTITY_MSU', 'PROMO_DURATION', 'NETINV_LC']].values))
    
    final_df.loc[idx] = new_row
  
  final_df['YEARWEEK'] = pd.to_datetime(final_df['YEARWEEK'])
  final_df.set_index('YEARWEEK', inplace = True)
  return final_df

In [0]:
def polish_promo_duration(imp_cust_brand_df):
  
  data = imp_cust_brand_df.copy()
  #data['PROMO_DURATION_WEEKS'] = np.ceil(data['PROMO_DURATION'] / 7)
  data['NORM_PROMO_SPEND'] = np.zeros(data.shape[0])
  data['IS_PROMOTED'] = np.zeros(data.shape[0])
  
  for date, row in data.iterrows():
    
    if (any([promo_duration > 0 for promo_duration in row['PROMO_DURATION']])):
      
      for pr_spend, pr_dur in zip(row['PROMO_SPEND'], row['PROMO_DURATION']):
        num_weeks = np.ceil(pr_dur / 7)

        if (num_weeks > 0):
          end_date = date + np.timedelta64(7 * int(num_weeks - 1), 'D')
          total_qty = data.loc[date:end_date, 'QUANTITY_MSU'].sum()
            # 1. Promo has spend 1000, duration 5 weeks, sales 50, 100, 20, 10, 50
            # 2. calculate total sales during the promo. 50 + 100 + 20 + 10 + 50 -> total_qty
            # 3. For each week, get the qty share. week 1, 50 / total_qty, week 2 100/totalqty
            # 4. promo spend for that week is total promo spend * qty share
          data.loc[date:end_date, 'NORM_PROMO_SPEND'] += pr_spend * data.loc[date:end_date, 'QUANTITY_MSU'] / (total_qty + 1)
          #data.loc[date:end_date, 'NORM_PROMO_SPEND'] += pr_spend / num_weeks
          data.loc[date:end_date, 'IS_PROMOTED'] = 1

          #data.loc[date:end_date, ['IS_PROMOTED', 'PROMO_SPEND']] = [1, row['PROMO_SPEND'] / ((data.loc[date:end_date, 'QUANTITY_MSU'] + 1) * num_weeks)]
          #data.loc[date:end_date, ['IS_PROMOTED', 'PROMO_SPEND']] = [1, row['PROMO_SPEND'] / (data.loc[date:end_date, 'QUANTITY_MSU'].sum() + 1)]
  
  #data['IS_PROMOTED'] = data['IS_PROMOTED'].astype('category')
  data.rename(columns = {'PROMO_SPEND': 'OLD_PROMO_SPEND', 
                        'NORM_PROMO_SPEND': 'PROMO_SPEND'}, 
             inplace = True)
  
  return data

In [0]:
def get_discount(df):
  
  df['PRICE_PER_CASE'] = df['NETINV_LC'] / (df['QUANTITY_MSU'] + 1)
  base_price = df['PRICE_PER_CASE'].max()
  df['DISCOUNT'] = (base_price - df['PRICE_PER_CASE']) * 100 / base_price
  
  return df

# Get important Pairs

In [0]:
total_value = data['NETINV_LC'].sum()
total_cases = data['QUANTITY_MSU'].sum()
cust_ean_grp = data.groupby(['CUSTOMER', 'EAN']).sum()[['NETINV_LC', 'QUANTITY_MSU']]
cust_ean_grp['NETINV_LC_SHARE'] = cust_ean_grp['NETINV_LC'] * 100 / total_value
cust_ean_grp['QUANTITY_MSU_SHARE'] = cust_ean_grp['QUANTITY_MSU'] * 100 / total_cases
cust_ean_grp['NETINV_LC_CUMSUM'] = cust_ean_grp['NETINV_LC_SHARE'].sort_values(ascending = False).cumsum(axis = 0)
cust_ean_grp['QUANTITY_MSU_CUMSUM'] = cust_ean_grp['QUANTITY_MSU_SHARE'].sort_values(ascending = False).cumsum(axis = 0)
cust_ean_grp.sort_values('NETINV_LC_SHARE', ascending = False).head()

Out[39]:

NETINV_LC 
 QUANTITY_MSU 
 NETINV_LC_SHARE 
 QUANTITY_MSU_SHARE 
 NETINV_LC_CUMSUM 
 QUANTITY_MSU_CUMSUM 
 
 
 CUSTOMER 
 EAN 
 
 
 
 
 
 
 
 
 
 
 40069503 
 17751493006955 
 21358.03 
 139.0 
 100.0 
 100.0 
 100.0 
 100.0 
 
 
 40033990 
 17702425679211 
 0.00 
 0.0 
 0.0 
 0.0 
 100.0 
 100.0 
 
 
 40162584 
 17751493007693 
 0.00 
 0.0 
 0.0 
 0.0 
 100.0 
 100.0 
 
 
 40169289 
 17702425543789 
 0.00 
 0.0 
 0.0 
 0.0 
 100.0 
 100.0 
 
 
 40173338 
 17751493001141 
 0.00 
 0.0 
 0.0 
 0.0 
 100.0 
 100.0

In [0]:
cust_ean_grp.shape

Out[40]: (10, 6)

In [0]:
## Get top pairs
top_pairs = cust_ean_grp.copy()
top_25_pairs = top_pairs.sort_values('NETINV_LC_SHARE', ascending = False).copy()
top_25_pairs['NETINV_LC'].sum() * 100 / total_value
top_25_pairs.reset_index(inplace = True)
top_25_pairs_list_full = list(top_25_pairs[['CUSTOMER', 'EAN']].to_records(index = False))

# Get pair dictionary
top_25_pairs_list = top_25_pairs_list_full
# [len(top_pairs_list_full)]
# top_25_pairs_dict = {}
 
# for cust, ean in top_25_pairs_list:
#   if (cust in top_25_pairs_dict):
#     top_25_pairs_dict[cust].append(ean)
#   else:
#     top_25_pairs_dict[cust] = [ean]

# Build final models

In [0]:
def build_cust_ean_model(df_full, lag, ma, trend, forecast_steps = 12, start_date = datetime.datetime(2019, 7, 1), 
                         exp_bl_ratio = 0.5):
  '''
  df: customer-ean pair
  '''
  df_full.loc[df_full['PROMO_SPEND'] < 0, 'PROMO_SPEND'] = 0
  df_full.loc[df_full['QUANTITY_MSU'] < 0, 'QUANTITY_MSU'] = 0
  df_full.loc[df_full['NETINV_LC'] < 0, 'NETINV_LC'] = 0
  df_full = impute_data(df_full)
  df_full = polish_promo_duration(df_full)
  df = df_full[df_full.index >= start_date]
  df = get_discount(df)
  
  y = df['QUANTITY_MSU']
  X_train = df[['PROMO_SPEND', 'DISCOUNT']][:-forecast_steps]
  y_train = y[:-forecast_steps]
  X_test = df[['PROMO_SPEND', 'DISCOUNT']][-forecast_steps:]
  
  # Build topline model with optimal hyperparameters
  model = SARIMAX(endog = y_train, exog = X_train, order = (lag, 0, [ma]), 
                    trend = trend, enforce_stationarity = True, initialization='approximate_diffuse').fit(disp = False, maxiter = 250)
  
  # Predict baseline values
  bl_df = df.copy()
  bl_df['PROMO_SPEND'] = 0
  bl_df['DISCOUNT'] = 0
  bl_df['IS_PROMOTED'] = 0
  
  # Add in weeks left over in 2021 to finaL_df
  # Get last date in final_df -> last_date
  last_time = bl_df.index.values[-1]
  curr_date = last_time + np.timedelta64(7, 'D')

  # Add rows with exogenous values = 0 for every 7 days after last_date until year becomes > 2021
  while (int(curr_date.astype(str)[:4]) <= 2022):
    bl_df.loc[curr_date] = [0] * bl_df.shape[1]
    bl_df.loc[curr_date, 'QUANTITY_MSU'] = -1
    curr_date += np.timedelta64(7, 'D')
  
  bl_endog = bl_df['QUANTITY_MSU']
  bl_exog = bl_df[['PROMO_SPEND', 'DISCOUNT']]
  bl_model = SARIMAX(endog = bl_endog, exog = bl_exog, order = (lag, 0, [ma]), 
                    trend = trend, enforce_stationarity = True, initialization='approximate_diffuse')
  bl_model = bl_model.filter(model.params)
  
  bl_fc_obj = bl_model.get_prediction(start = 0, dynamic = lag)
  bl_preds = bl_fc_obj.predicted_mean
  bl_conf_int = bl_fc_obj.conf_int(alpha = 0.2)
  bl_preds.loc[bl_preds < 0] = 0
  bl_conf_int.loc[bl_conf_int['lower QUANTITY_MSU'] < 0, 'lower QUANTITY_MSU'] = 0
  bl_conf_int.loc[bl_conf_int['upper QUANTITY_MSU'] < 0, 'upper QUANTITY_MSU'] = 0
  
  bl_preds_ma = bl_preds.copy()
  low_ci_diff = bl_preds_ma - bl_conf_int['lower QUANTITY_MSU']
  high_ci_diff = bl_conf_int['upper QUANTITY_MSU'] - bl_preds_ma
  window = 8
  for idx in range(len(bl_preds_ma)):
    if (idx >= window - 1):
      bl_preds_ma[idx] = bl_preds_ma[idx - window + 1:idx + 1].mean()
      #bl_conf_int[idx] = bl_conf_int[idx - window + 1:idx + 1].mean()
      bl_conf_int['lower QUANTITY_MSU'][idx] = bl_preds_ma[idx] - low_ci_diff[idx - window + 1:idx + 1].mean()
      bl_conf_int['upper QUANTITY_MSU'][idx] = bl_preds_ma[idx] + high_ci_diff[idx - window + 1:idx + 1].mean()
      #bl_conf_int[idx] = bl_conf_int[idx - window + 1:idx + 1].mean()
  bl_conf_int.loc[bl_preds_ma == 0, 'lower QUANTITY_MSU'] = bl_preds_ma.median() - low_ci_diff.loc[bl_preds_ma == 0]
  bl_conf_int.loc[bl_preds_ma == 0, 'upper QUANTITY_MSU'] = bl_preds_ma.median() + high_ci_diff.loc[bl_preds_ma == 0]
  bl_preds_ma.replace(0, bl_preds_ma.median(), inplace = True)
  
  bl_conf_int.loc[bl_conf_int['lower QUANTITY_MSU'] < 0, 'lower QUANTITY_MSU'] = 0
  bl_conf_int.loc[bl_conf_int['upper QUANTITY_MSU'] < 0, 'upper QUANTITY_MSU'] = 0
  bl_preds_ma.replace(0, bl_preds_ma.median(), inplace = True)
  
  max_date = bl_preds_ma.index.max()
  for date in df_full.index:
    
    if (date not in bl_preds_ma.index):
      #print ('Impute for date {}'.format(date))
      curr_date = date
      curr_yearweek = curr_date.strftime('%W')
      bl_values_list = []
      low_ci_values = []
      high_ci_values = []
      
      while (curr_date < max_date):
        #print ('Curr date: {}'.format(curr_date))
        next_year_date = datetime.datetime.strptime(str(int(curr_date.strftime('%Y')) + 1) + str(curr_yearweek) + '-1', "%Y%W-%w")
        #print ('Next date: {}'.format(next_year_date))
        if (next_year_date > max_date):
          break
        bl_values_list.append(bl_preds_ma.loc[next_year_date])
        low_ci_values.append(bl_preds_ma.loc[next_year_date] - bl_conf_int.loc[next_year_date, 'lower QUANTITY_MSU'])
        high_ci_values.append(bl_conf_int.loc[next_year_date, 'upper QUANTITY_MSU'] - bl_preds_ma.loc[next_year_date])
        curr_date = next_year_date
      
      bl_preds_ma.loc[date] = sum(bl_values_list) / len(bl_values_list)
      bl_conf_int.loc[date, 'lower QUANTITY_MSU'] = bl_preds_ma.loc[date] - (sum(low_ci_values) / len(low_ci_values))
      bl_conf_int.loc[date, 'upper QUANTITY_MSU'] = (sum(high_ci_values) / len(high_ci_values)) + bl_preds_ma.loc[date]
      #print ('Imputed value for date {}: {}'.format(date, bl_preds_ma.loc[date]))
      
    else:
      break
  
  bl_conf_int.loc[bl_conf_int['lower QUANTITY_MSU'] < 0, 'lower QUANTITY_MSU'] = 0
  bl_conf_int.loc[bl_conf_int['upper QUANTITY_MSU'] < 0, 'upper QUANTITY_MSU'] = 0
  actuals_for_bl = pd.concat([df_full.loc[df_full.index < start_date, 'QUANTITY_MSU'], bl_df['QUANTITY_MSU']])
  actuals_for_bl.sort_index(inplace = True)
  bl_preds_ma.sort_index(inplace = True)
  
  # Visualization topline mode - only with the endog
  endog_model = SARIMAX(endog = df_full['QUANTITY_MSU'], order = (lag, 0, [ma]), 
                    trend = trend, enforce_stationarity = True, initialization='approximate_diffuse').fit(disp = False, maxiter = 250)
  endog_pred_obj = endog_model.get_prediction(dynamic = 0)
  endog_in_sample_pred = endog_pred_obj.predicted_mean
  
  actuals_for_pred = df_full['QUANTITY_MSU']
  num_extra_steps = 0
  last_time = actuals_for_pred.index.max()
  curr_date = last_time + np.timedelta64(7, 'D')
  while (curr_date.year <= 2022):
    actuals_for_pred.loc[curr_date] = -1
    curr_date += np.timedelta64(7, 'D')
    num_extra_steps += 1
  
  endog_fc_obj = endog_model.get_forecast(steps = num_extra_steps)
  endog_pred = endog_fc_obj.predicted_mean
  endog_pred = pd.concat([endog_in_sample_pred, endog_pred], axis = 0)
  
  # Set negative values to 0
  endog_pred.loc[endog_pred < 0] = 0

  # Capping
  bl_preds_ma = simulate_baseline(bl_preds_ma, actuals_for_bl, exp_bl_ratio)

  return endog_pred, actuals_for_pred, bl_preds_ma, actuals_for_bl, bl_conf_int

In [0]:
def simulate_baseline(bl_preds_ma, actuals_for_bl, exp_bl_ratio):
    
  vis_dates = actuals_for_bl.index[(actuals_for_bl.index >= datetime.datetime(2020, 1, 1))]
  vis_bl_preds = bl_preds_ma[vis_dates]
  vis_actuals = actuals_for_bl[vis_dates]
  future_actuals_idx = actuals_for_bl.index[actuals_for_bl == -1]
  vis_bl_ratio = vis_bl_preds.loc[~ vis_bl_preds.index.isin(future_actuals_idx)].sum() / vis_actuals.loc[~ vis_actuals.index.isin(future_actuals_idx)].sum()
  #vis_actuals.loc[future_actuals_idx] = endog_pred.loc[future_actuals_idx]

  min_date = bl_preds_ma.index.min()
  for future_date in future_actuals_idx:
    values_list = []
    prev_year_date = datetime.datetime.strptime(str(future_date.isocalendar()[0] - 1) + str(future_date.isocalendar()[1]) + '-1', 
                                       '%Y%W-%w')
    while (prev_year_date >= min_date):
        if (actuals_for_bl.loc[prev_year_date] != -1):
            values_list.append(actuals_for_bl.loc[prev_year_date])
        prev_year_date = datetime.datetime.strptime(str(prev_year_date.isocalendar()[0] - 1) + str(prev_year_date.isocalendar()[1]) + '-1', 
                                       '%Y%W-%w')
    if (len(values_list) > 0):
        #print ('Future date {}:\n\tValues {}'.format(future_date, values_list))
        vis_actuals.loc[future_date] = sum(values_list) / len(values_list)
    else:
        vis_actuals.loc[future_date] = 0
    
  
  #print ('Current {} - Expected {}'.format(vis_bl_ratio, exp_bl_ratio))
  #print ('\nOld Vis BL Values: \n{}'.format(vis_bl_preds))
    
  if (vis_bl_ratio > (exp_bl_ratio + 0.1 * exp_bl_ratio)):
        #print ('Higher')
        adj_bl_preds = 1.1 * (vis_actuals * exp_bl_ratio)
        
  elif (vis_bl_ratio < (exp_bl_ratio - 0.1 * exp_bl_ratio)):
        #print ('Lower')
        adj_bl_preds = 0.9 * (vis_actuals * exp_bl_ratio)
        
  else:
    #print ('Same')
    adj_bl_preds = vis_bl_preds.copy()
    
  diff_thresh = 0.2
  diff_bl_preds = (vis_bl_preds - adj_bl_preds).abs() / (adj_bl_preds + 1)
  #print ('\nRelative Difference: \n{}'.format(diff_bl_preds.loc[diff_bl_preds > 0]))
  #print ('Number of rows to be altered - {}\nPercentage of rows to be altered - {}'
   #      .format(vis_bl_preds.loc[diff_bl_preds > diff_thresh].shape[0], 
    #            vis_bl_preds.loc[diff_bl_preds > diff_thresh].shape[0] * 100 / vis_bl_preds.shape[0]))
  vis_bl_preds.loc[diff_bl_preds > diff_thresh] = adj_bl_preds.loc[diff_bl_preds > diff_thresh]
  #print ('\nNew Vis BL Values: \n{}'.format(vis_bl_preds.loc[diff_bl_preds > 0]))
    
  bl_preds_ma.loc[vis_dates] = vis_bl_preds.copy()
  new_vis_bl_ratio = vis_bl_preds.loc[~ vis_bl_preds.index.isin(future_actuals_idx)].sum() / vis_actuals.loc[~ vis_actuals.index.isin(future_actuals_idx)].sum()
  print ('Old {} - Current {} - Expected {}'.format(vis_bl_ratio, new_vis_bl_ratio, exp_bl_ratio))

  return bl_preds_ma

In [0]:
def build_result_table(out_dict):
  
  df = pd.DataFrame.from_dict({(i,j,k): out_dict[i][j][k] 
                           for i in out_dict.keys() 
                           for j in out_dict[i].keys()
                           for k in out_dict[i][j].keys()},
                       orient='index')
  df.reset_index(inplace = True)
  df.rename(columns = {'level_0': 'CUSTOMER', 
                      'level_1': 'EAN', 
                      'level_2': 'CAL_WEEK'}, 
           inplace = True)
  
  return df

In [0]:
def build_average_model(df, exp_bl_ratio):
  
  df_full = df.copy()
  df_full = impute_data(df_full)
  non_zero_mask = df_full['QUANTITY_MSU'] > 0
  actuals = df_full['QUANTITY_MSU']
  
  if (not any(non_zero_mask)):
    mean_sales = 0
  else:
    mean_sales = df_full['QUANTITY_MSU'][non_zero_mask].mean()
    
  bl_preds = df_full.copy()
  bl_preds['QUANTITY_MSU'] = mean_sales
  bl_preds = bl_preds['QUANTITY_MSU']
  
  tl_preds = df_full.copy()
  tl_preds['QUANTITY_MSU'] = 0
  tl_preds = tl_preds['QUANTITY_MSU']
  
  last_time = df_full.index.max()
  curr_date = last_time + np.timedelta64(7, 'D')

  # Add rows with exogenous values = 0 for every 7 days after last_date until year becomes > 2021
  while (curr_date.year <= 2021):
    bl_preds.loc[curr_date] = mean_sales
    tl_preds.loc[curr_date] = 0
    actuals.loc[curr_date] = -1
    curr_date += np.timedelta64(7, 'D')
    
  bl_conf_int = pd.DataFrame(np.zeros((bl_preds.shape[0], 2)), columns = ['lower QUANTITY_MSU', 'upper QUANTITY_MSU'])
  bl_conf_int.set_index(bl_preds.index, inplace = True)
    
  bl_preds = simulate_baseline(bl_preds, actuals, exp_bl_ratio)
  
  return tl_preds, actuals, bl_preds, actuals, bl_conf_int

In [0]:
def process_bl_ratio_sheet(sheet):
    sheet['Sold_To'] = sheet['Sold_To'].astype(int).astype(str)
    
    return sheet

In [0]:
def build_models_wrapper(df, hp_table, cust_ean_grp, forecast_steps = 12, exp_bl_ratio_sheet_path = pd.DataFrame()):
  '''
  df: the entire dataset
  '''
  
  out_dict = {}
  bl_dict = {}
  
  for idx, row in enumerate(cust_ean_grp.itertuples()):
  
    try:
      cust, ean = row[1], row[2]
      print ('\n-------- Row {}, Customer {}, OSKU {} --------'.format(idx + 1, cust, ean))
      #pair_df = df[(df['CUSTOMER'] == cust) & (df['EAN'] == ean)]
      #print ('Date range: {} - {}\nNumber of rows: {}\nNumber of rows with non-zero actuals: {}'
       #      .format(pair_df['YEARWEEK'].min(), pair_df['YEARWEEK'].max(), pair_df['YEARWEEK'].shape[0], pair_df[pair_df['QUANTITY_MSU'] > 0].shape[0])
        #    )
    
      ean_category = df.loc[df['EAN'] == ean, 'SECTOR'].unique()[0].replace(' ', '_')
      plan_cust = df.loc[df['CUSTOMER'] == cust, 'PLAN_LEVEL'].unique()[0]
      plan_cust = str(int(plan_cust[:plan_cust.find('-')]))
      #print ('Sector {}, Planning Customer {}'.format(ean_category, plan_cust))
      exp_bl_ratio = 0.5  
        
      if (exp_bl_ratio_sheet_path.shape[0] > 0):
        
          bl_ratio_df = process_bl_ratio_sheet(exp_bl_ratio_sheet_path)
          bl_ratio_sheet_cust_list = bl_ratio_df['Sold_To'].unique() 
        
          if (ean_category in bl_ratio_df.columns):

                if (plan_cust in bl_ratio_sheet_cust_list):
                    df_bl_ratio = round(bl_ratio_df.loc[bl_ratio_df['Sold_To'] == plan_cust, ean_category].values[0], 2)
                else:
                    df_bl_ratio = round(bl_ratio_df.loc[bl_ratio_df['Sold_To'] == '0', ean_category].values[0], 2)
                
                exp_bl_ratio = df_bl_ratio if df_bl_ratio > 0 else exp_bl_ratio
            
      #print ('Expected BL Ratio of {}'.format(exp_bl_ratio))
    
      if (cust not in out_dict):
              out_dict[cust] = {}
              bl_dict[cust] = {}
      hp_list = hp_table[(hp_table['Customer'] == cust) & (hp_table['OSKU'] == ean)]

      if (hp_list.shape[0] > 0):
          print ('Found hyperparameters')
          cust, ean, lag, ma, trend, tr_mape, val_mape, bl_ratio, end_date, start_date = hp_list.values[0]
          trend = [int(x) for x in trend if x not in '[,] ']
          if (start_date == 0):
            start_date = datetime.datetime(2018, 1, 1)

          cust_ean_df = df[(df['CUSTOMER'] == cust) & (df['EAN'] == ean)]
          lag = int(lag)
          ma = int(ma)
          tr_data_end_date = end_date - datetime.timedelta(weeks = 12)
          #forecast_steps = len(cust_ean_df[cust_ean_df['YEARWEEK'] > tr_data_end_date])

          forecast_steps = 0
          curr_date = tr_data_end_date + np.timedelta64(7, 'D')
          while (curr_date <= cust_ean_df['YEARWEEK'].max()):
            curr_date += np.timedelta64(7, 'D')
            forecast_steps += 1
            
          if (forecast_steps == 0):
              forecast_steps = 12

          #trend = [int(x) for x in trend if x not in '[,]']
          tl_preds, actuals_for_preds, bl_preds, actuals_for_bl, bl_ci = build_cust_ean_model(cust_ean_df, lag, ma, trend, 
                                                                                                forecast_steps, start_date, 
                                                                                             exp_bl_ratio = exp_bl_ratio)
      else:
        print ('Not found in the hyperparameter table')
        try:
            forecast_steps = 12
            cust_ean_df = df[(df['CUSTOMER'] == cust) & (df['EAN'] == ean)]
            start_date = datetime.datetime(2018, 1, 1)
            tl_preds, actuals_for_preds, bl_preds, actuals_for_bl, bl_ci = build_cust_ean_model(cust_ean_df, 4, 3, [0], 
                                                                                                forecast_steps, start_date, 
                                                                                               exp_bl_ratio = exp_bl_ratio)
        except Exception as e:
            exception_type, exception_object, exception_traceback = sys.exc_info()
            line_number = exception_traceback.tb_lineno
            print ('Model building error: {} - {}'.format(line_number, e))
            print ('Building average model')
            tl_preds, actuals_for_preds, bl_preds, actuals_for_bl, bl_ci = build_average_model(cust_ean_df, exp_bl_ratio)
            
    except Exception as e:
      exception_type, exception_object, exception_traceback = sys.exc_info()
      line_number = exception_traceback.tb_lineno
      print ('{} - {}'.format(line_number, e))
      continue

    out_dict[cust][ean] = {
      date:{
        'predicted': pred, 
        'actual': actual,
      } for (date, pred, actual) in zip(tl_preds.index, tl_preds, actuals_for_preds)
    }

    bl_dict[cust][ean] = {
      date:{
        'baseline': bl, 
        'actual': actual, 
        'low_ci': low_ci, 
        'high_ci': high_ci
      } for (date, bl, actual, low_ci, high_ci) in zip(bl_preds.index, bl_preds, actuals_for_bl, bl_ci['lower QUANTITY_MSU'], bl_ci['upper QUANTITY_MSU'])
    }
    
  pred_out_df = build_result_table(out_dict)
  bl_out_df = build_result_table(bl_dict)
  
  bl_out_df.loc[bl_out_df['low_ci'] < 0, 'low_ci'] = 0
  bl_out_df.loc[bl_out_df['high_ci'] < 0, 'high_ci'] = 0
  
  return pred_out_df, bl_out_df

In [0]:
# output = build_models_wrapper(cap_data, hp_table)
copy_cust_ean_grp = cust_ean_grp.copy()
copy_cust_ean_grp.sort_values('NETINV_LC_SHARE', ascending = False, inplace = True)
copy_cust_ean_grp.reset_index(inplace = True)
# test_cust_ean_grp = copy_cust_ean_grp[(copy_cust_ean_grp['CUSTOMER'] == test_cust) & 
#                                       (copy_cust_ean_grp['EAN'] == test_ean)]
#test_cust_ean_grp = copy_cust_ean_grp[(copy_cust_ean_grp['CUSTOMER'] == test_cust) & 
 #                                     (copy_cust_ean_grp['EAN'].isin(test_ean_list))]
pred_out_df, bl_out_df = build_models_wrapper(data[data['YEARWEEK'] <= datetime.datetime.now()], hp_table, 
                                              copy_cust_ean_grp[:], exp_bl_ratio_sheet_path = bl_ratio_sheet)
# cap_data.to_csv('Peru_capdata.csv')

-------- Row 1, Customer 40069503, OSKU 17751493006955 --------
Found hyperparameters
69 - zero-size array to reduction operation maximum which has no identity

-------- Row 2, Customer 40033990, OSKU 17702425679211 --------
Not found in the hyperparameter table
Model building error: 78 - zero-size array to reduction operation maximum which has no identity
Building average model
Old nan - Current nan - Expected 0.5

-------- Row 3, Customer 40162584, OSKU 17751493007693 --------
Not found in the hyperparameter table
Model building error: 78 - zero-size array to reduction operation maximum which has no identity
Building average model
Old nan - Current nan - Expected 0.5

-------- Row 4, Customer 40169289, OSKU 17702425543789 --------
Not found in the hyperparameter table
Model building error: 78 - zero-size array to reduction operation maximum which has no identity
Building average model
Old nan - Current nan - Expected 0.5

-------- Row 5, Customer 40173338, OSKU 17751493001141 --------
Not found in the hyperparameter table
Model building error: 78 - zero-size array to reduction operation maximum which has no identity
Building average model
Old nan - Current nan - Expected 0.5

-------- Row 6, Customer 40174192, OSKU 17751493000823 --------
Not found in the hyperparameter table
Model building error: 78 - zero-size array to reduction operation maximum which has no identity
Building average model
Old nan - Current nan - Expected 0.6499999761581421

-------- Row 7, Customer 40176283, OSKU 17751493009147 --------
Not found in the hyperparameter table
Model building error: 78 - zero-size array to reduction operation maximum which has no identity
Building average model
Old nan - Current nan - Expected 0.5

-------- Row 8, Customer 40176491, OSKU 17751493008492 --------
Not found in the hyperparameter table
Model building error: 78 - zero-size array to reduction operation maximum which has no identity
Building average model
Old nan - Current nan - Expected 0.5

-------- Row 9, Customer 40176857, OSKU 17751493006962 --------
Not found in the hyperparameter table
Model building error: 78 - zero-size array to reduction operation maximum which has no identity
Building average model
Old nan - Current nan - Expected 0.5

-------- Row 10, Customer 40179626, OSKU 17751493008416 --------
Not found in the hyperparameter table
Model building error: 78 - zero-size array to reduction operation maximum which has no identity
Building average model
Old nan - Current nan - Expected 0.699999988079071

In [0]:
spark.createDataFrame(pred_out_df).write.mode("errorifexists").saveAsTable("default.peru_data_pred_actuals")
spark.createDataFrame(bl_out_df).write.mode("errorifexists").saveAsTable("default.peru_data_bl_actuals")

# Join output files

In [0]:
bl_out_table = pd.DataFrame()
bl_out_table = bl_out_df
  
bl_out_table.head()

Out[53]:

CUSTOMER 
 EAN 
 CAL_WEEK 
 baseline 
 actual 
 low_ci 
 high_ci 
 
 
 
 
 0 
 40033990 
 17702425679211 
 2019-07-22 
 0 
 0.0 
 0.0 
 0.0 
 
 
 1 
 40033990 
 17702425679211 
 2019-07-29 
 0 
 -1.0 
 0.0 
 0.0 
 
 
 2 
 40033990 
 17702425679211 
 2019-08-05 
 0 
 -1.0 
 0.0 
 0.0 
 
 
 3 
 40033990 
 17702425679211 
 2019-08-12 
 0 
 -1.0 
 0.0 
 0.0 
 
 
 4 
 40033990 
 17702425679211 
 2019-08-19 
 0 
 -1.0 
 0.0 
 0.0

In [0]:
bl_out_table.groupby(['CUSTOMER', 'EAN']).sum().shape

Out[54]: (9, 4)

In [0]:
pred_out_table = pd.DataFrame()
pred_out_table = pred_out_df
  
pred_out_table.head()

Out[55]:

CUSTOMER 
 EAN 
 CAL_WEEK 
 predicted 
 actual 
 
 
 
 
 0 
 40033990 
 17702425679211 
 2019-07-22 
 0 
 0.0 
 
 
 1 
 40033990 
 17702425679211 
 2019-07-29 
 0 
 -1.0 
 
 
 2 
 40033990 
 17702425679211 
 2019-08-05 
 0 
 -1.0 
 
 
 3 
 40033990 
 17702425679211 
 2019-08-12 
 0 
 -1.0 
 
 
 4 
 40033990 
 17702425679211 
 2019-08-19 
 0 
 -1.0

In [0]:
pred_out_table.groupby(['CUSTOMER', 'EAN']).sum().shape

Out[56]: (9, 2)

In [0]:
spark.createDataFrame(pred_out_table).write.mode("overwrite").saveAsTable("default.peru_data_pred_actuals_all")
spark.createDataFrame(bl_out_table).write.mode("overwrite").saveAsTable("default.peru_data_bl_actuals_all")